In [134]:
libraries = c("dplyr", "tidyverse", "data.table", "readxl")
for(x in libraries) {library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE)}

theme_set(theme_bw())

In [135]:
path <- "../data/flight/all_region/"

options(warn=-1)
list.files(path = path, pattern = "*xlsx") -> file_list
read_xlsx_files <- function(x){
    df <- read_xlsx(path = paste0(path, x), skip = 5)
    df[-1, c(1,2,27)] -> df_2019
    colnames(df_2019) <- c("destination", "series", "v2019")
    return(df_2019)}
lapply(file_list, read_xlsx_files ) -> df_list

for(i in 1:length(df_list)){
    colnames(df_list[[i]]) <- c("destination", "series", substr(file_list[i],1,nchar(file_list[i])-5))
}

options(warn=0)

New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
•

New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`


In [136]:
merge.all <- function(x, y) {merge(x, y, all=TRUE, by=c("destination", "series"))}
output <- Reduce(merge.all, df_list)

In [137]:
output %>% filter(!(destination %in% c("END", "Türkiye", "Czechia"))) %>%
mutate(destination=case_when(destination==c("Curaçao")~c("Curacao"), 
                             destination==c("Côte d\'Ivoire")~c("Cote d'Ivoire"), 
                             destination==c("Taiwan Province of China")~c("Taiwan, Province of China"), 
                             destination==c("Saint Vincent and the Grenadines")~c("Saint Vincent and The Grenadines"),
                             destination==c("Palestine")~c("Palestine, State of"),
                             TRUE~destination)) -> output

In [138]:
new_list <- list()
unique(output$destination) -> country_list

for (g in 1:length(country_list)){
    output %>% filter(destination==country_list[g]) -> temp
    if(nrow(temp)>=2){
        if(any(temp$series==c("VFR"))){temp %>% filter(series==c("VFR")) -> new_list[[g]]}
        else if(any(temp$series==c("VFN")) && !(temp$series %in% c("VFR")))
        {temp %>% filter(series==c("VFN")) -> new_list[[g]]}
        else if(any(temp$series==c("TFR")) && !(temp$series %in% c("VFR", "VFN")))
        {temp %>% filter(series==c("TFR")) -> new_list[[g]]}
        else if(any(temp$series==c("TFN")) && !(temp$series %in% c("VFR", "VFR", "TFR")))
        {temp %>% filter(series==c("TFN")) -> new_list[[g]]}
        else if(any(temp$series==c("TCER")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN")))
        {temp %>% filter(series==c("TCER")) -> new_list[[g]]}
        else if(any(temp$series==c("TCEN")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN", "TCER")))
        {temp %>% filter(series==c("TCEN")) -> new_list[[g]]}
        else if(any(temp$series==c("THSR")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN", "TCER", "TCEN")))
        {temp %>% filter(series==c("THSR")) -> new_list[[g]]}
        else if(any(temp$series==c("THSN")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN", "TCER", "TCEN", "THSR")))
        {temp %>% filter(series==c("THSN")) -> new_list[[g]]}
    }
    else{temp -> new_list[[g]]}
}
            
do.call("rbind", new_list) -> flight_matrix
# flight_matrix %>% 
write.csv(flight_matrix, "../data/flight/flight_matrix.csv")     

In [139]:
setdiff(flight_matrix$destination, colnames(flight_matrix[,3:215]))

[1] "Cabo Verde"                      "Eswatini"                       
[3] "French Guiana"                   "Micronesia, Federated States of"
[5] "Reunion"                         "Sint Maarten (Dutch part)"      
[7] "Taiwan, Province of China"       "Tanzania, United Republic of"

In [140]:
flight_matrix$destination

[1] "Albania"                           "Algeria"                          
  [3] "American Samoa"                    "Andorra"                          
  [5] "Angola"                            "Anguilla"                         
  [7] "Antigua and Barbuda"               "Argentina"                        
  [9] "Armenia"                           "Aruba"                            
 [11] "Australia"                         "Austria"                          
 [13] "Azerbaijan"                        "Bahamas"                          
 [15] "Bahrain"                           "Bangladesh"                       
 [17] "Barbados"                          "Belarus"                          
 [19] "Belgium"                           "Belize"                           
 [21] "Benin"                             "Bermuda"                          
 [23] "Bhutan"                            "Bolivia, Plurinational State of"  
 [25] "Bosnia and Herzegovina"            "Botswana"                         
 [27] "Brazil"                            "Brunei Darussalam"                
 [29] "Bulgaria"                          "Burkina Faso"                     
 [31] "Cabo Verde"                        "Cambodia"                         
 [33] "Cameroon"                          "Canada"                           
 [35] "Cayman Islands"                    "Central African Republic"         
 [37] "Chile"                             "China"                            
 [39] "Colombia"                          "Comoros"                          
 [41] "Congo"                             "Congo, Democratic Republic of the"
 [43] "Cook Islands"                      "Costa Rica"                       
 [45] "Croatia"                           "Cuba"                             
 [47] "Curacao"                           "Cyprus"                           
 [49] "Czech Republic"                    "Cote d'Ivoire"                    
 [51] "Denmark"                           "Dominica"                         
 [53] "Dominican Republic"                "Ecuador"                          
 [55] "Egypt"                             "El Salvador"                      
 [57] "Estonia"                           "Eswatini"                         
 [59] "Ethiopia"                          "Fiji"                             
 [61] "Finland"                           "France"                           
 [63] "French Guiana"                     "French Polynesia"                 
 [65] "Gambia"                            "Georgia"                          
 [67] "Germany"                           "Greece"                           
 [69] "Grenada"                           "Guam"                             
 [71] "Guatemala"                         "Guinea"                           
 [73] "Guyana"                            "Haiti"                            
 [75] "Honduras"                          "Hong Kong, China"                 
 [77] "Hungary"                           "Iceland"                          
 [79] "India"                             "Indonesia"                        
 [81] "Iran, Islamic Republic of"         "Ireland"                          
 [83] "Israel"                            "Italy"                            
 [85] "Jamaica"                           "Japan"                            
 [87] "Jordan"                            "Kazakhstan"                       
 [89] "Kenya"                             "Kiribati"                         
 [91] "Korea, Republic of"                "Kuwait"                           
 [93] "Kyrgyzstan"                        "Lao People's Democratic Republic" 
 [95] "Latvia"                            "Lebanon"                          
 [97] "Lesotho"                           "Liechtenstein"                    
 [99] "Lithuania"                         "Luxembourg"                       
[101] "Macao, China"                      "Madagascar"                       
[103] "Malawi"          

In [141]:
colnames(flight_matrix[,3:215])

[1] "Afghanistan"                          
  [2] "Albania"                              
  [3] "Algeria"                              
  [4] "American Samoa"                       
  [5] "Andorra"                              
  [6] "Angola"                               
  [7] "Anguilla"                             
  [8] "Antigua and Barbuda"                  
  [9] "Argentina"                            
 [10] "Armenia"                              
 [11] "Aruba"                                
 [12] "Australia"                            
 [13] "Austria"                              
 [14] "Azerbaijan"                           
 [15] "Bahamas"                              
 [16] "Bahrain"                              
 [17] "Bangladesh"                           
 [18] "Barbados"                             
 [19] "Belarus"                              
 [20] "Belgium"                              
 [21] "Belize"                               
 [22] "Benin"                                
 [23] "Bermuda"                              
 [24] "Bhutan"                               
 [25] "Bolivia, Plurinational State of"      
 [26] "Bosnia and Herzegovina"               
 [27] "Botswana"                             
 [28] "Brazil"                               
 [29] "British Virgin Islands"               
 [30] "Brunei Darussalam"                    
 [31] "Bulgaria"                             
 [32] "Burkina Faso"                         
 [33] "Burundi"                              
 [34] "Cambodia"                             
 [35] "Cameroon"                             
 [36] "Canada"                               
 [37] "Cape Verde"                           
 [38] "Cayman Islands"                       
 [39] "Central African Republic"             
 [40] "Chad"                                 
 [41] "Chile"                                
 [42] "China"                                
 [43] "Colombia"                             
 [44] "Comoros"                              
 [45] "Congo, Democratic Republic of the"    
 [46] "Congo"                                
 [47] "Cook Islands"                         
 [48] "Costa Rica"                           
 [49] "Cote d'Ivoire"                        
 [50] "Croatia"                              
 [51] "Cuba"                                 
 [52] "Curacao"                              
 [53] "Cyprus"                               
 [54] "Czech Republic"                       
 [55] "Denmark"                              
 [56] "Djibouti"                             
 [57] "Dominica"                             
 [58] "Dominican Republic"                   
 [59] "Ecuador"                              
 [60] "Egypt"                                
 [61] "El Salvador"                          
 [62] "Equatorial Guinea"                    
 [63] "Eritrea"                              
 [64] "Estonia"                              
 [65] "Ethiopia"                             
 [66] "Fiji"                                 
 [67] "Finland"                              
 [68] "France"                               
 [69] "French Polynesia"                     
 [70] "Gabon"                                
 [71] "Gambia"                               
 [72] "Georgia"                              
 [73] "Germany"                              
 [74] "Ghana"                                
 [75] "Greece"                               
 [76] "Grenada"                              
 [77] "Guadeloupe"                           
 [78] "Guam"                                 
 [79] "Guatemala"                            
 [80] "Guinea-Bissau"                        
 [81] "Guinea"                               
 [82] "Guyana"                               
 [83] "Haiti"                                
 [84] "Honduras"                             
 [85] "Hong Kong, China"                     
 [86] "Hungary"                              
 [87] "Iceland"                              
